# Automated ML


In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="creditcardfraud")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: workspace-rainel
Azure region: brazilsouth
Subscription id: 611bccaf-ced7-4b1d-9395-57559c451c39
Resource group: raineldias88-rg


## Dataset

### Overview
I am using credit card transactions data from Kaggle ( more details on readme file). Problem statement is to predict whether the transaction is fraud (1) or not (0). This is a supervised binary classification task. 

In [2]:
from azureml.core import Dataset
dataset = Dataset.get_by_name(ws, name='creditcard')
df = dataset.to_pandas_dataframe()

In [3]:
#Check the data frame by running head
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
## check the stats
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284806.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.841366,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.958151e-16,-3.147640e-15,...,1.471982e-16,8.042109e-16,5.282450e-16,4.458267e-15,1.426896e-15,1.701640e-15,-3.671606e-16,-1.218152e-16,88.349619,0.001727
std,47488.228330,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.250000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84691.500000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.750000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [5]:
# check how output column is distributed

df['Class'].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

#### As we can see above that the data is heavily imbalanced, there is below 1% fraud cases, so I will be using AUC weighted metric for evaluation instead of using just the accuracy.

In [6]:
# Using pandas profiling package to get exploratory data analysis (EDA) report on HTML format.
# The report is added to my repo as output_profile, this captures detailed EDA and descriptive statistics on data

from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file='output_profile.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Getting the Data Ready for Modeling

In [7]:
# check if compute clusters are set up
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "compute-cluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Splitting the data in training and test, and uploading dataset to the workspace for AutoML 

In [8]:
## Split the Training data and test data
y = df.iloc[:,-1].values   # output variable
X = df.iloc[:, :-1].values  # feature variables

In [9]:
#split the data set into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
import pandas as pd
df_train = pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train)], axis=1)

In [11]:
df_train.shape # merged training data to be passed to automl

(227845, 31)

In [12]:
columns= df.columns

In [13]:
df_train.columns= columns

In [14]:
df_train['Class'] = df_train['Class'].astype(bool)

In [15]:
df_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,143352.0,1.955041,-0.380783,-0.315013,0.330155,-0.509374,-0.086197,-0.627978,0.035994,1.054560,...,0.238197,0.968305,0.053208,-0.278602,-0.044999,-0.216780,0.045168,-0.047145,9.99,False
1,117173.0,-0.400975,-0.626943,1.555339,-2.017772,-0.107769,0.168310,0.017959,-0.401619,0.040378,...,-0.153485,0.421703,0.113442,-1.004095,-1.176695,0.361924,-0.370469,-0.144792,45.90,False
2,149565.0,0.072509,0.820566,-0.561351,-0.709897,1.080399,-0.359429,0.787858,0.117276,-0.131275,...,-0.314638,-0.872959,0.083391,0.148178,-0.431459,0.119690,0.206395,0.070288,11.99,False
3,93670.0,-0.535045,1.014587,1.750679,2.769390,0.500089,1.002270,0.847902,-0.081323,0.371579,...,0.063525,0.443431,-0.072754,0.448192,-0.655203,-0.181038,-0.093013,-0.064931,117.44,False
4,82655.0,-4.026938,1.897371,-0.429786,-0.029571,-0.855751,-0.480406,-0.435632,1.313760,0.536044,...,-0.480691,-0.230369,0.250717,0.066399,0.470787,0.245335,0.286904,-0.322672,25.76,False


In [16]:
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

def_blob_store = ws.get_default_datastore()
Dataset.Tabular.register_pandas_dataframe(df_train, target= def_blob_store, name= "df_train_tabular", description=None, tags=None, show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/11bbceed-75f0-40ed-b9b9-3ff2956827d6/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


{
  "source": [
    "('workspaceblobstore', 'managed-dataset/11bbceed-75f0-40ed-b9b9-3ff2956827d6/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "068abf42-312c-4a90-a7e9-84ee5e60a968",
    "name": "df_train_tabular",
    "version": 1,
    "workspace": "Workspace.create(name='workspace-rainel', subscription_id='611bccaf-ced7-4b1d-9395-57559c451c39', resource_group='raineldias88-rg')"
  }
}

In [17]:
dataset = Dataset.get_by_name(ws, name='df_train_tabular')
dataset.to_pandas_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,143352.0,1.955041,-0.380783,-0.315013,0.330155,-0.509374,-0.086197,-0.627978,0.035994,1.054560,...,0.238197,0.968305,0.053208,-0.278602,-0.044999,-0.216780,0.045168,-0.047145,9.99,False
1,117173.0,-0.400975,-0.626943,1.555339,-2.017772,-0.107769,0.168310,0.017959,-0.401619,0.040378,...,-0.153485,0.421703,0.113442,-1.004095,-1.176695,0.361924,-0.370469,-0.144792,45.90,False
2,149565.0,0.072509,0.820566,-0.561351,-0.709897,1.080399,-0.359429,0.787858,0.117276,-0.131275,...,-0.314638,-0.872959,0.083391,0.148178,-0.431459,0.119690,0.206395,0.070288,11.99,False
3,93670.0,-0.535045,1.014587,1.750679,2.769390,0.500089,1.002270,0.847902,-0.081323,0.371579,...,0.063525,0.443431,-0.072754,0.448192,-0.655203,-0.181038,-0.093013,-0.064931,117.44,False
4,82655.0,-4.026938,1.897371,-0.429786,-0.029571,-0.855751,-0.480406,-0.435632,1.313760,0.536044,...,-0.480691,-0.230369,0.250717,0.066399,0.470787,0.245335,0.286904,-0.322672,25.76,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227840,75618.0,1.173488,0.100792,0.490512,0.461596,-0.296377,-0.213165,-0.165254,0.119221,-0.114199,...,-0.186027,-0.574283,0.161405,-0.006140,0.091444,0.109235,-0.020922,0.003967,1.98,False
227841,159000.0,-0.775981,0.144023,-1.142399,-1.241113,1.940358,3.912076,-0.466107,1.360620,0.400697,...,0.037078,-0.019575,0.241830,0.682820,-1.635109,-0.770941,0.066006,0.137056,89.23,False
227842,79795.0,-0.146609,0.992946,1.524591,0.485774,0.349308,-0.815198,1.076640,-0.395316,-0.491303,...,0.052649,0.354089,-0.291198,0.402849,0.237383,-0.398467,-0.121139,-0.196195,3.94,False
227843,87931.0,-2.948638,2.354849,-2.521201,-3.798905,1.866302,2.727695,-0.471769,2.217537,0.580199,...,-0.332759,-1.047514,0.143326,0.678869,0.319710,0.426309,0.496912,0.335822,1.00,False


## AutoML Configuration

Exit criteria is to run autoML for 30 mins to save time and resources. Since the data is imbalanced I am picking AUC weighted metric. When computing the weighted area under the ROC curve, weights vary with the values of the true positive rate (TPrate) among regions in a bid to focus on the accuracy of minority class that is more important in common.

In [18]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "experiment_timeout_minutes": 30,
    "primary_metric": 'AUC_weighted'
}


automl_config = AutoMLConfig(
    task='classification',
    training_data= dataset,
    label_column_name='Class',
    n_cross_validations=5,
    compute_target=cpu_cluster,
    **automl_settings
)

## Run Details

In [19]:
# Submitting the Experiment

from azureml.widgets import RunDetails
remote_run = exp.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
creditcardfraud,AutoML_2bd69a7a-1b23-48eb-a4f3-d6068a5628c5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
creditcardfraud,AutoML_2bd69a7a-1b23-48eb-a4f3-d6068a5628c5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest clas

{'runId': 'AutoML_2bd69a7a-1b23-48eb-a4f3-d6068a5628c5',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-11-19T10:50:56.059166Z',
 'endTimeUtc': '2022-11-19T11:40:01.708902Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"creditcardfraud","subscription_id":"611bccaf-ced7-4b1d-9395-57559c451c39","resource_group":"raineldias88-rg","workspace_name":"workspace-rainel","region":"brazilsouth","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"

## Best Model

In [20]:
# Getting the best model

best_run, fittedb_model = remote_run.get_output()

print(best_run)

Run(Experiment: creditcardfraud,
Id: AutoML_2bd69a7a-1b23-48eb-a4f3-d6068a5628c5_20,
Type: azureml.scriptrun,
Status: Completed)


In [21]:
# checking the accuracy of model
best_run.get_metrics()['AUC_weighted']

0.9791951184894012

In [25]:
# Saving the Best Model in .pkl format, and registering the model

import joblib

description = "AutoML model trained on Credit Card Fraud Dataset"

os.makedirs('outputs', exist_ok=True)
joblib.dump(fittedb_model, filename="outputs/automl-model.pkl")
automl_model = remote_run.register_model(model_name='automl-creditcard', description=description)

## Model Deployment

The accuracy of AutoML is higher than Hyper Drive method, so I am choosing this autoML model to deploy

In [26]:
# Deploying Model as Azure Container Instance service

from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Predict if a transaction is Fraud (1) or not (0)',
    auth_enabled=True)

In [27]:
from azureml.core.model import Model
from azureml.automl.core.shared import constants
from azureml.core.model import InferenceConfig


model = Model(ws, 'automl-creditcard')
myenv = best_run.get_environment()
entry_script = 'score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

inference_config = InferenceConfig(entry_script=entry_script, environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='automl-creditcard-1', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-11-19 11:56:41+00:00 Creating Container Registry if not exists..
2022-11-19 12:06:41+00:00 Registering the environment.
2022-11-19 12:06:42+00:00 Use the existing image..
2022-11-19 12:06:42+00:00 Submitting deployment to compute..
2022-11-19 12:06:50+00:00 Checking the status of deployment automl-creditcard-1..
2022-11-19 12:09:18+00:00 Checking the status of inference endpoint automl-creditcard-1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


### Getting two data points from Test set to make it json ready for scoring

In [28]:
#getting test data ready
test_data = pd.DataFrame(X_test[:2]) #pick two data points for test
features = columns[:-1]
test_data.columns= features
test_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,41505.00,-16.53,8.58,-18.65,9.51,-13.79,-2.83,-16.70,7.52,-8.51,...,-1.51,1.19,-1.13,-2.36,0.67,-1.41,-0.46,-2.02,-1.04,364.19
1,44261.00,0.34,-2.74,-0.13,-1.39,-1.45,1.02,-0.52,0.22,0.90,...,0.51,-0.21,-0.94,-0.53,-1.16,0.31,-0.75,0.04,0.10,520.12


In [29]:
test_data['json'] = test_data.apply(lambda x: x.to_json(), axis=1)

In [30]:
test_data['json'][0]


'{"Time":41505.0,"V1":-16.5265065691,"V2":8.5849717959,"V3":-18.6498531852,"V4":9.5055935151,"V5":-13.7938185271,"V6":-2.8324042994,"V7":-16.701694296,"V8":7.5173439037,"V9":-8.5070586368,"V10":-14.1101844415,"V11":5.2992363496,"V12":-10.8340064815,"V13":1.6711202533,"V14":-9.3738585836,"V15":0.3608056416,"V16":-9.8992465408,"V17":-19.2362923698,"V18":-8.3985519949,"V19":3.1017353689,"V20":-1.5149234353,"V21":1.1907386948,"V22":-1.127670009,"V23":-2.3585787698,"V24":0.673461329,"V25":-1.4136996746,"V26":-0.4627623614,"V27":-2.0185752488,"V28":-1.0428041697,"Amount":364.19}'

In [31]:
test_data['json'][1]

'{"Time":44261.0,"V1":0.3398120639,"V2":-2.7437452373,"V3":-0.134069511,"V4":-1.3857293091,"V5":-1.4514133205,"V6":1.0158865939,"V7":-0.5243790569,"V8":0.2240603761,"V9":0.8997460049,"V10":-0.5650116836,"V11":-0.0876702573,"V12":0.9794269879,"V13":0.0768828168,"V14":-0.2178838121,"V15":-0.1368295877,"V16":-2.1428920902,"V17":0.1269560647,"V18":1.7526615075,"V19":0.4325462237,"V20":0.5060438852,"V21":-0.2134358436,"V22":-0.9425250246,"V23":-0.5268191745,"V24":-1.1569918974,"V25":0.3112105102,"V26":-0.7466466791,"V27":0.0409958027,"V28":0.1020378246,"Amount":520.12}'

In [32]:
# print the scoring uri info 
print("Health_Status: "+ service.state)
print("Key: " + service.get_keys()[0])
print("Scoring URI: " + service.scoring_uri)


Health_Status: Healthy
Key: twnjY5J7CLsm9ag2wBhzlARilolExocw
Scoring URI: http://d268e639-bcbc-4b57-be21-b48640ac7421.brazilsouth.azurecontainer.io/score


### Add these json formatted test data points to the endpoint.py file which will be passed for scoring

In [33]:
%run endpoint.py

{"result": [true, false]}


## Print logs and Delete Resources

In [34]:
## Lets enable app insights logging
service.update(enable_app_insights=True)

In [37]:
# print the logs
print(service.get_logs())

2022-11-19T12:23:13,991220932+00:00 - iot-server/run 
2022-11-19T12:23:13,991287531+00:00 - rsyslog/run 
2022-11-19T12:23:14,003051078+00:00 - gunicorn/run 
2022-11-19T12:23:14,008855103+00:00 | gunicorn/run | 
2022-11-19T12:23:14,011838364+00:00 | gunicorn/run | ###############################################
2022-11-19T12:23:14,016950197+00:00 | gunicorn/run | AzureML Container Runtime Information
2022-11-19T12:23:14,020862946+00:00 - nginx/run 
2022-11-19T12:23:14,024843294+00:00 | gunicorn/run | ###############################################
2022-11-19T12:23:14,026438374+00:00 | gunicorn/run | 
2022-11-19T12:23:14,028035853+00:00 | gunicorn/run | 
2022-11-19T12:23:14,046499812+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20220930.v4
2022-11-19T12:23:14,049419974+00:00 | gunicorn/run | 
2022-11-19T12:23:14,055813191+00:00 | gunicorn/run | 
2022-11-19T12:23:14,060283433+00:00 | gunicorn/run | PATH environment variable: /azureml-env

In [38]:
# deleting the webservice
service.delete()